## 1. Linear Regression using Gradiant Decent algorithm

In linear regression there are $n$ input features and $1$ output feature that we want to predict. 
So visually what we're trying to do, is that we want to find a line \_n-dimensional\_ that "fits" data points. or more presicely, is as close as possible to all data points.

The description above gives us an idea about what we're looking for but it can't be considered as a problem statement.



### 1.1 What is the problem?
Making a clear problem statement is as important as solving the problem.
for that matter, let's note that we can model our input features with a vector $x^{(i)}$ in a vector space $V$ and output is a scaler $y^{(i)}$ in the field $F$ in which we define the vector space. here $V = \mathbb{R}^n$ and $F = \mathbb{R}$ and superscript $(i)$ indicates sample index.


so we can form a matrix $X$ which contains all of sample vectors ($x^{(i)}$):
$$
X_{ij} = x^{(i)}_j
$$

Now, let's assume that there is a function $h^*: V \rightarrow F$ that maps these points to a scaler and is the function that exactly "fits" all datapoints. So $h^*(x^{(i)})$. Obviously it's not neccesserily linear or any other form.

We define $h$ as the *hypothesis* \_an estimation of $h^*$\_ given the constraints that $h$ is a linear function.

As it's known that every linear function can be represented with a vector of coefficients, the problem of finding $h$ is equivalent to finding it's vector of coefficients,
which is represented by $\Theta = [\theta_{1},\dots, \theta_{n}]$ . So it's convinient to write $h_{\Theta}$ instead of just $h$.

now we're ready to write the problem statement.

#### statement 1:
> Given value of $h^*$ for m points/vectors $x^{(1)}, \dots,x^{(m)}$,  find a linear function $h_{\Theta}$ which estimates $h^*$.

From the terms "as close as possible" (1.) and "estimates" (1.2) it's not very clear what we should do. In order to define a better metric for that, which means how good some function $h_{\Theta}$ is we define a cost function $J : V \rightarrow F$ as:


Its convenient to summorize $h_\Theta(x^{(i)})$ as $\hat{y}^{(i)}$, so:
$$ J(\Theta) = \frac{1}{2m} \sum_{i=1}^m (\hat{y}^{(i)}-y^{(i)})^2 $$
So we want to find some $h_{\Theta}$ which minimizes the value of $J$.

now we can update the problem statement as:

#### statement 2:
> Given value of $h^*$ for m points/vectors $x^{(1)}, \dots,x^{(m)}$,  find a linear function $h_{\Theta}$ for which $J(\Theta)$ is minimized.


In [9]:
def h(self, x):
    res = 0
    # add fake feature
    x.append(1)
    for i in range(self.n):
        res += x[i]*self.theta[i]
    return res

def J(self):
    res = 0
    for i in range(self.m):
        res += (self.h(self.X[i])-self.y[i])**2
    res /= 2*self.m
    return res

### 1.2 Solution
We use gradiant decent algorithm to find some $\Theta$ which is local optimum for $J$.

This is an overview on how this algorithm works:

Let's assume we have $\Theta_{1}$ as first hypothesis. we can initialize this to some random vector.

We claim that $J(\Theta_{2}) \leq J(\Theta_{1})$ for $\Theta_{2} = \Theta_{1} - \eta \nabla J(\Theta_{1})$.

Doing this $p-1$ times we'll end up with a seqence $\Theta_{1}, \dots, \Theta_{p}$ and each one is a better estimation than the previous one.


Next we calculate gradiant of cost funcion this way:
$$
\nabla J(\Theta) = [\frac{\partial}{\partial \theta_{1}} J, \dots,  \frac{\partial}{\partial \theta_{n}} J]
$$


$$
\frac{\partial}{\partial \theta_{j}} J = \frac{1}{m} \sum_{i=1}^m(\hat{y}^{(i)} - y^{i})x^{(i)}_j
$$

we can calculate sum of these terms and show that:
$$\nabla J = Y^T X$$
where $ Y = \frac{1}{m}[\hat{y}^{(1)} - y^{(1)}, \dots, \hat{y}^{(m)}- y^{(m)}] $

In [10]:
def gradiant(self):
    gradiant_vector = [0]*self.n
    for j in range(self.n):
        for i in range(self.m):
            gradiant_vector[j] += (self.h(self.X[i])-self.y[i])*self.X[i][j]
        gradiant_vector[j] /= self.m
    return gradiant_vector
    
def gradiant_decent(self):
    for i in range(self.p):
        for j in range(self.n):
            self.theta[j] -= self.nabla*self.gradiant()[j]


### 1.3 Wrap up
This is everything put together in Regression class

In [1]:
# import libraries
import time
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

In [195]:
class Regression:
    
    def __init__(self, number_of_iterations, learning_rate):
        self.number_of_iterations = number_of_iterations
        self.learning_rate = learning_rate
        
    def h(self, x):
        return self.theta.dot(x)

    def J(self):
        res = 0
        for i in range(self.n):
            res += (self.h(self.X[i])-self.Y[i])**2
        res /= 2*self.n
        return res

    def gradiant(self):
        nabla = np.array(self.n)
        Y = np.zeros(self.m)
        for i in range(self.m):
            Y[i] = (self.h(self.X[i])-self.y[i])/self.m
        nabla = Y.dot(self.X)
        if (np.linalg.norm(nabla) != 0):
            nabla /= np.linalg.norm(nabla)
        return nabla
        
    def gradiant_decent(self):
        for k in range(self.number_of_iterations):
            nabla = self.gradiant()
            for j in range(self.n):
                self.theta[j] -= self.learning_rate*nabla[j]
            
    def run(self, input_dataset, output_dataset):
        self.X = np.array(input_dataset)
        self.y = np.array(output_dataset)
        
        # m(number of data), n(number of features or dimentions)
        self.m = self.X.shape[0]
        self.n = self.X.shape[1]
        self.theta = np.ones(self.n)
        print(self.n, self.m)

        self.gradiant_decent()

    def test(self, X_test, Y_test):
        X_test = np.array(X_test)
        y_hat = np.zeros(len(X_test))
        for i in range(len(X_test)):
            y_hat[i] = self.h(X_test[i])
            
        print(self.theta)
        print(y_hat)
        print(mean_absolute_error(y_hat, Y_test))
        print((mean_squared_error(y_hat, Y_test))**0.5)
        print(r2_score(y_hat, Y_test))

### 1.4 Testing with dataset
Next, we use a dataset to test everything

In [196]:
df = pd.read_csv("~/downloads/Flight_Price_Dataset_Q2.csv")
# print(df.head())
departure_time_mapping = {
    "Early_Morning": 4,
    "Morning": 3,
    "Afternoon": 2,
    "Night": 1, 
    "Late_Night": 0
}
stops_mapping = {
    "zero": 2,
    "one": 1,
    "two_or_more": 0
}
class_mapping = {
    "Economy": 0,
    "Business": 1
}
df["departure_time"] = df["departure_time"].map(departure_time_mapping)
df["stops"] = df["stops"].map(stops_mapping)
df["arrival_time"] = df["arrival_time"].map(departure_time_mapping)
df["class"] = df["class"].map(class_mapping)
df["duration"] = (50 - df["duration"]) / 10
df["days_left"] = df["days_left"] / 10

df = df.dropna()
df = df.reset_index(drop=True)
Y = df["price"]
X = df.drop("price", axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

X_test["fake_feature"] = 1
X_train["fake_feature"] = 1
# print(X_train.head())
# print(X_train.value_counts('duration'))
# print(X_train['duration'].min(), X_train['duration'].max())
# print(X_train.value_counts('days_left'))
# print(X_train['days_left'].min(), X_train['days_left'].max())

In [205]:
learning_rate = 0.01
number_of_iterations = 1000

start_time = time.time()
# reg = Regression(number_of_iterations, learning_rate)
# x = [
#     [4, 1],
#     [9, 3],
#     [19, 8],
# ]
# y = [2, 6, 16]
# reg.run(x, y)

reg.run(X_train, Y_train)
print("Training Time: %s seconds" % round(time.time() - start_time, 6))

7 122194
Training Time: 147.369536 seconds


In [206]:
# xt = [
#     [9, 4],
#     [2, 2],
#     [13, 5],
# ]
# yt = [8, 4, 10]
# reg.test(xt, yt)
reg.test(X_test, Y_test)

[5.54759561 2.8325589  4.16962846 2.37576285 7.4569713  5.3826233
 2.76690916]
[81.73483421 72.67262773 54.45878435 ... 60.34540564 66.81885484
 78.78713066]
19967.78459402524
29877.497401459783
-6058599.025501116


In [199]:
regr = LinearRegression()
regr.fit(X_train.values.tolist(), Y_train.values.tolist())
Y_pred = regr.predict(X_test.values.tolist())
print(mean_absolute_error(Y_test.values.tolist(), Y_pred))
print((mean_squared_error(Y_test.values.tolist(), Y_pred))**0.5)
print(r2_score(Y_test.values.tolist(), Y_pred))

4624.881513241347
7148.265547735831
0.896542234050758
